### Welcome to Week 6 Day 3!

Let's experiment with a bunch more MCP Servers

In [1]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio
import os
from IPython.display import Markdown, display
from datetime import datetime
load_dotenv(override=True)

True

### The first type of MCP Server: runs locally, everything local

Here's a really interesting one: a knowledge-graph based memory.

It's a persistent memory store of entities, observations about them, and relationships between them.

https://github.com/modelcontextprotocol/servers/tree/main/src/memory


---


So, the first type of MCP server that I mentioned is one that you run locally, and everything it does is local. Again, remembering, we downloaded it from a public repo, somewhere public, but we're going to run it. Server will be applied locally. And we're going to pick another JavaScript one, a node one, and I want to pick this one, which is great. It is a way to give our agent memory, and to give it knowledge graph-based memory, a special kind of memory that understands about entities and observations about those entities and relationships between those entities. Memory is a hot topic, and people often ask me questions about memory, and they think of memory like it's kind of one construct, perhaps because LangChain thinks of it that way, but like there is one memory. But in this MCP age, we think of memory just as being another set of tools that we can equip our LLM with. And in fact, it could have many different types of tools relating to memory. This in particular is giving it the ability to store information about things, observations, and relationships. We could give it this, and we could give it other kinds of context. All of this ends up just being information that it can request by tools or that becomes available in its prompts. And so they're just different techniques to give the LLM more context. Anyway, so these are the parameters. Remember, you will start by specifying the parameters of your MCP server. And in this case, it's MPX because we're running a node version, and we're going to be running this version called libsql, which is a version, a memory that stores to a SQL-like database. I've actually given a link here to one that this is closely based on, which is one that gives you access to a JSON file, but doesn't actually store it to SQL. And I found that this is a little bit less stable. This is a better version of it that I prefer. It also allows you to specify a directory and a name for your memory. And this allows you to have different memory stores for different agents. So I find that quite handy. So I'm saying that look in somewhere called memory and look for a database called ed. And I believe it might already exist. It does. So why don't we just delete this so that it doesn't have any memory to start with? And move that to trash, starting from nothing. So if you have ed in a folder there, then delete it. If you don't have a folder, then create one called memory.


In [2]:
params = {"command": "npx","args": ["-y", "mcp-memory-libsql"],"env": {"LIBSQL_URL": "file:./memory/ed.db"}}

async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as server:
    mcp_tools = await server.list_tools()

mcp_tools

[Tool(name='create_entities', description='Create new entities with observations and optional embeddings', inputSchema={'type': 'object', 'properties': {'entities': {'type': 'array', 'items': {'type': 'object', 'properties': {'name': {'type': 'string'}, 'entityType': {'type': 'string'}, 'observations': {'type': 'array', 'items': {'type': 'string'}}, 'embedding': {'type': 'array', 'items': {'type': 'number'}, 'description': 'Optional vector embedding for similarity search'}}, 'required': ['name', 'entityType', 'observations']}}}, 'required': ['entities']}, annotations=None),
 Tool(name='search_nodes', description='Search for entities and their relations using text or vector similarity', inputSchema={'type': 'object', 'properties': {'query': {'oneOf': [{'type': 'string', 'description': 'Text search query'}, {'type': 'array', 'items': {'type': 'number'}, 'description': 'Vector for similarity search'}]}}, 'required': ['query']}, annotations=None),
 Tool(name='read_graph', description='Get 

All right. So first of all, let's just find out what tools do we have associated with this. We have tools create entities, search nodes, read graph, create relations, delete entity, and delete relation. So we're allowing it to build this kind of connectivity between the things it wants to remember.


So now we have instructions. You use your entity tools as persistent memory to store and record information about your conversations. My name is ed. I'm an LLM engineer. I'm teaching a course about agents, including the incredible MCP protocol. And I say something about MCP. 

In [3]:
instructions = "You use your entity tools as a persistent memory to store and recall information about your conversations."
request = "My name's Ed. I'm an LLM engineer. I'm teaching a course about AI Agents, including the incredible MCP protocol. \
MCP is a protocol for connecting agents with tools, resources and prompt templates, and makes it easy to integrate AI agents with capabilities."
model = "gpt-4.1-mini"

And I give it a model. And let's see how this gets on. So as before, this is hopefully a pattern that you're quite familiar with. We use this context manager. We have MCP server studio, which we know is going to create the MCP client. We pass in the parameters. We give this timeout of 30 seconds. And then, as usual, we create our agent with instructions, with the model, and now with the MCP server. And now we call runner.run with the agent and our request. And let's see what happens. Armed with these tools, is it able to take this piece of information about me and do something with it? 

In [4]:
async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as mcp_server:
    agent = Agent(name="agent", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("conversation"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

Nice to meet you, Ed! It sounds like your course on AI Agents is very exciting, especially with the inclusion of the MCP protocol. If you'd like, I can help you prepare course materials, examples, or answer questions about AI agents and the MCP protocol. How can I assist you today?

So it says, nice to meet you, Ed. It's great to know you're teaching a course about AI agents and covering MCP protocol, blah, blah, blah, blah, blah. And if you look over on the left, it has created a file, `memory/ed.db` 

---

So now for the follow-up question. We're coming back. We're creating another agent. We're giving it instructions. We're asking for the same model, and we're giving it the same MCP server with access to the memory. And we're saying, my name's Ed. What do you know about me? And we'll see what this MCP server is able to do.

In [5]:
async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as mcp_server:
    agent = Agent(name="agent", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("conversation"):
        result = await Runner.run(agent, "My name's Ed. What do you know about me?")
    display(Markdown(result.final_output))

I currently don't have any information stored about you, Ed. Would you like to share some details about yourself? I can store that information for future reference.

 I know that you're Ed, an LLM engineer. We teach you a course about AI agents, and in this course, we teach you about the MCP protocol. So there you go. This is a quick, simple example, but it shows you how we were so easily able to equip our agent with a memory that's able to handle relationships between things. And so this is just a great starting example, and we'll be using this memory in the future as well, and it could be handy for any of your projects too. And of course, it's always a good idea to check the trace, which we should do. Come on in. 
 
 ![](../img/88.png)
 
 And conversation with our agent is right here, and you will see that it called the search nodes. It looked into the query for Ed, and it got back here JSON structure that reflects the entity type, the observations, and so on. And so you can look through this and get a sense for how the memory works, and you can use this to try and build up some more sophisticated memory about the different parts of your conversation that you want your agent to remember. Okay. Now we're going to go on to another MCP tool, and this time it's going to be the second type, which is the type that runs locally but uses the Internet, which we've already done with Vetch and with Playbytes. But let's do another, and a handy one.

### Check the trace:

https://platform.openai.com/traces

### The 2nd type of MCP server - runs locally, calls a web service

### Brave Search - apologies - this will need another API key! But it's free again.

https://brave.com/search/api/

Set up your account, and put your key in the .env under `BRAVE_API_KEY`

So there's been many times in the last few weeks that we have used tools that do internet searches. You probably remember the hosted tool with OpenAI that we used right back in week two, which was quite expensive. Was it two and a half cents for every single web search? And we've used, I think, Tavli, and we've used Serpa. Well, we're going to do another one. So sorry to make you set up another key, but this one's a really great one. It's called Brave Search, and it's a company that specializes in API-driven search, including for use with AI. And it's free again. At least I think you get 2,000 free searches a month. So it's very generous, and I have set it up. I've actually got a paid account because I've been using it a lot, but you very much can stay on the free version. I've got a link here for where you go to set up your account. If I go up here, it brings up the Brave Search. You sign up, you go through, and get an API key super easily without needing any credit card or anything like that. So once you've done that, you come back, you take your Brave API key, and you put it in your .env file as usual, and then you'll have to rerun load .env up at the top.

In [6]:
load_dotenv(override=True)

True

So then what we do is we collect our Brave API key, and this time you'll notice that when we specify the params for our MCP server, we're passing in these environment variables, these settings as well. So that's a new twist for this one, so that we can tell Brave about our API key. And it's another node, another JavaScript MCP server, so we run it this way. And in fact, this is one of Anthropix out-of-the-box reference implementations. So this is part of Anthropix offerings, the Brave Search. And so again, even though we're running online searches, the MCP server is going to be running on my box. So the code for this is remote, is online. We are downloading that code and running it locally using MPX. So it starts online, we bring it locally, we run it locally. And then of course, what that code actually does is it makes a call to a web service offered by the company Brave, and it passes in our key to that service. So that's why this is architecture two, MCP server running locally, calling out to the cloud. And I'm probably belaboring this. You're probably like, yeah, I get it, I get it, enough. Okay, so MCP server studio, again, is how we do this. We pass in the parameters. Let's just see what tools we get with this. We get a Brave web search tool, which performs a web search with the API. And then a local search, searches for local businesses. I think I only get that because I'm paying the pay plan. I don't think that comes with the free plan, actually. So again, the key difference of something like Fetch is that with Fetch, we're running a browser locally, and we're just going to a web address. With Brave, we're making a web call to Brave to ask it to run like a Google search, but it's not using Google, it's using its own search engine, and return the results of that query.

In [7]:
# Primero verificar que la clave se cargó
load_dotenv(override=True)
brave_api_key = os.getenv("BRAVE_API_KEY")
print(f"BRAVE_API_KEY loaded: {brave_api_key is not None}")

if brave_api_key:
    env = {"BRAVE_API_KEY": brave_api_key}
    params = {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-brave-search"], "env": env}
    
    async with MCPServerStdio(params=params) as server:
        mcp_tools = await server.list_tools()
    print("Brave Search tools:", mcp_tools)
else:
    print("BRAVE_API_KEY not set. Skipping Brave Search MCP server.")

BRAVE_API_KEY loaded: True
Brave Search tools: [Tool(name='brave_web_search', description='Performs a web search using the Brave Search API, ideal for general queries, news, articles, and online content. Use this for broad information gathering, recent events, or when you need diverse web sources. Supports pagination, content filtering, and freshness controls. Maximum 20 results per request, with offset for pagination. ', inputSchema={'type': 'object', 'properties': {'query': {'type': 'string', 'description': 'Search query (max 400 chars, 50 words)'}, 'count': {'type': 'number', 'description': 'Number of results (1-20, default 10)', 'default': 10}, 'offset': {'type': 'number', 'description': 'Pagination offset (max 9, default 0)', 'default': 0}}, 'required': ['query']}, annotations=None), Tool(name='brave_local_search', description="Searches for local businesses and places using Brave's Local Search API. Best for queries related to physical locations, businesses, restaurants, services,

And so we're going to ask a question, you're able to search the web for information, research the latest news on the Amazon stock price, and we give it the current date, and then we run the server right here.

In [8]:
env = {"BRAVE_API_KEY": os.getenv("BRAVE_API_KEY")}
params = {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-brave-search"], "env": env}

async with MCPServerStdio(params=params) as server:
    mcp_tools = await server.list_tools()

mcp_tools

[Tool(name='brave_web_search', description='Performs a web search using the Brave Search API, ideal for general queries, news, articles, and online content. Use this for broad information gathering, recent events, or when you need diverse web sources. Supports pagination, content filtering, and freshness controls. Maximum 20 results per request, with offset for pagination. ', inputSchema={'type': 'object', 'properties': {'query': {'type': 'string', 'description': 'Search query (max 400 chars, 50 words)'}, 'count': {'type': 'number', 'description': 'Number of results (1-20, default 10)', 'default': 10}, 'offset': {'type': 'number', 'description': 'Pagination offset (max 9, default 0)', 'default': 0}}, 'required': ['query']}, annotations=None),
 Tool(name='brave_local_search', description="Searches for local businesses and places using Brave's Local Search API. Best for queries related to physical locations, businesses, restaurants, services, etc. Returns detailed information including:\

In [9]:
instructions = "You are able to search the web for information and briefly summarize the takeaways."
request = f"Please research the latest news on Amazon stock price and briefly summarize its outlook. \
For context, the current date is {datetime.now().strftime('%Y-%m-%d')}"
model = "gpt-4o-mini"

Okay, here we go, running that. You'll see, as usual, we're passing in the MCP server, we've got the timeout set, and we're calling run.run, and back we get, of course, the results of running this web search with the current price of Amazon, apparently $217, and more information about it. And as always, we should go and check out the trace.

In [10]:
async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as mcp_server:
    agent = Agent(name="agent", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("conversation"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

Here's a brief summary of the latest news regarding Amazon's stock price and outlook as of August 15, 2025:

1. **Current Stock Status**: Amazon's stock (AMZN) saw a notable performance, gaining approximately 3.95% over the past five trading sessions following a positive earnings report. It has achieved a year-to-date gain of about 0.67% and is up around 32.91% over the past year.

2. **Market Response**: Despite the recent gains, there is ongoing market volatility. After the latest earnings report, there was an 8% decline in stock price which raised investor concerns regarding increasing costs due to tariffs on imports from China. This is likely to affect pricing strategies and could impact consumer demand.

3. **Investment Actions**: Notably, some hedge funds are adjusting their stakes, with Egerton Capital reducing its holdings by nearly 884,482 shares, while Diamond Hill Capital increased its stake, reflecting mixed investor sentiment.

4. **Price Forecast**: Analysts project that Amazon's stock price could range from a minimum of $201 to a maximum of $252 in August 2025, with an expected average price around $222.

Overall, while Amazon's recent performance shows resilience, the market's outlook remains cautious due to external economic pressures and investor adjustments.

### As usual, check out the trace:

Yep, it's calling a Brave web search. The query was Amazon stock price news, May 2025, and it got back these results. And so it seems to be working nicely end-to-end. That is an example of type two of the calling out to the web.

https://platform.openai.com/traces

## And now the third type: running remotely

And so now let's talk about type three of the MCP servers, the ones where the server itself, the MCP server, is running remotely, and you must connect to it using the SSE approach. And now here's the thing, as I say, it's not that common, and it's also quite flaky, because there's no guarantees that the people that you're connecting with keep their server running. I used to have an example in here of an MCP server that I found, a hosted MCP server, that you could connect to remotely, but it went down, and now you can't connect to it, and that's why I had to take out my code at the top. So it's no longer running, so that example didn't work. And so I tried to look for some other examples, and the other examples that I found are all for paid business services, where you are already paying for something, and it tends to be from some professional business plans.

It's actually really hard to find a "remote MCP server" aka "hosted MCP server" aka "managed MCP server".

It's not a common model for using or sharing MCP servers, and there isn't a standard way to discover remote MCP servers.

Anthropic lists some remote MCP servers, but these are for paid applications with business users:

https://docs.anthropic.com/en/docs/agents-and-tools/remote-mcp-servers

> So let me show you that. The Anthropic docs have a section on remote MCP servers, that's what we're talking about here. And it says several companies have deployed remote servers that developers can connect to. And so here are some examples of them, and you'll see why I'm not able to demo one of these, because I'd have to be a paying client. So Asana, if you are a client of Asana, Asana-like project management tool, then you could interact with your Asana workspace by using this remote MCP server, mcp.asana.com.sse. And then similarly for these others, like Intercom, like PayPal, if you use, not just PayPal for your personal PayPal, I have one of them, but I don't have commerce capabilities, I'm not a seller. And the others, and Square, and Mercato, and Zapier. I was going to try and set something up to show it, but you have to have a paid account. So it's honestly not common to be seeing this. It's only in these cases of like paid enterprise accounts where you don't want to run the MCP server yourself, you want to connect to it running on the cloud. And as I say, I really, I haven't seen much traction with this in the community. It's not like this is something that you come across often. And for many of these, for many of those commercial examples, you can run an MCP server locally as well, which is what people typically do.

CloudFlare has tooling for you to create and deploy your own remote MCP servers, but this does not seem to be a common practice:

https://developers.cloudflare.com/agents/guides/remote-mcp-server/

> Cloudflare, which was actually on that list, has some tools for you to create and run your own remote servers. If you're a Cloudflare customer and you use Cloudflare for your own websites, your own deployments, they have some really interesting sheets here which allow you to deploy a remote MCP server that other people could connect to. And they actually make it really quite easy. They walk you through the code here and how you can check in their screens that it's running. And then they give you, of course, the way that you would then connect to it from, say, desktop or indeed from your OpenAI Agents SDK. And this is then where you can add authentication, which is one of the hot new areas of MCP. If you are going to provide a hosted tool, then you would want to have a way that people can authenticate with it so that they can demonstrate they are who they say they are. But as I said one more time, and then I'll shut up because you're fed up of me saying this, but it's not a common way of doing things. And indeed, if you're running something like the Brave search that we just did, it would be more common for you to set up an API key with something like Brave and then pass in that API key when you call your local MCP server.


That's the more common configuration. And of course, this might change any day. Maybe managed hosted MCP servers will take off big time and I'll be easing my words. But as of right now, it's not terribly common. But you certainly could follow the Cloudflare instructions and set up your own should you wish. So that wraps up the third type. And we're now going to go back to the second type for a really cool example.

# And back to the 2nd type: the Polygon.io MCP Server

<table style="margin: 0; text-align: left; width:100%">
        <td>
            <h2 style="color:#ff7800;">PLEASE READ!!-</h2>
            <span style="color:#ff7800;">This service for financial market data has both a FREE plan and a PAID plan, and we can use either depending on your appetite.
            </span>
        </td>
</table>

## NEW SECTION: Introducing polygon.io

So the main project for this week is to set up a trading floor. How autonomous agents that can buy and sell equities, and they're not going to really buy and sell equities, they're going to be using simulated account management that we already built, but they will be using real market data. And there's a bunch of different MCP servers that can help provide you with market data. And I'll be playing with a bunch of them. My favorite one is the one provided by the company Polygon.io that's a very well-known professional provider of financial markets. And, yeah, I'll bring you to their website. Here they are. They're super well-known. And one of the big benefits that they have is that they have both a free plan and a paid plan. So I recommend you stick with the free plan. But if you're a sucker for this stuff like I am, then for sure you can spend a little somewhere between $20 and $30 a month to get more accurate market data. So with the free plan, you always get data as of the previous day's business close, stock prices as of yesterday's close. With the paid plan that I'm on, you get them on a 15-minute delay, but unlimited API use. And if you want to pay a lot more, you can get real-time market data. But there's no need for that unless you're already a trader. So this is Polygon. Please come in and sign up.

Polygon.io is a hugely popular financial data provider. It has a free plan and a paid plan. And it also has an MCP Server!

First, read up on polygon.io on their excellent website, including looking at their pricing:

https://polygon.io

### Polygon.io Part 1: Polygon.io free service (the paid will be totally optional, of course!)

1. Please sign up for polygon.io (top right)  
2. Once signed in, please select "Keys" in the left hand navigation
3. Press the blue "New Key" button
4. Copy the key name
5. Edit your .env file and add the row:

`POLYGON_API_KEY=xxxx`

In [11]:
load_dotenv(override=True)
polygon_api_key = os.getenv("POLYGON_API_KEY")
if not polygon_api_key:
    print("POLYGON_API_KEY is not set")

And once you're signed in, select the keys button in the left-hand navigation. You press the blue New Key button. And then you take that key and you put it in your .env file as a Polygon API key. And you go and just run this. And the fact that it didn't say Polygon API key is not set means that it was set. And then you can, for example, directly use the Polygon client. And they've got great docs on their website. And it's very easy to use. So this previous close for Apple's stock price, for example, will tell me that the previous close for Apple was $233.33

In [12]:
from polygon import RESTClient
client = RESTClient(polygon_api_key)
client.get_previous_close_agg("AAPL")[0]

PreviousCloseAgg(ticker='AAPL', close=232.78, high=235.12, low=230.85, open=234.055, timestamp=1755201600000, volume=51916275.0, vwap=232.7455)

### Wrapped into a python module that caches end of day prices

Okay, so I have written a little Python module called market.py that wraps some of these calls to Polygon. And I've done something a little bit sneaky. So the Polygon API, if you're on the free plan, is what they call rate-limited, which means you're only allowed to make that API call, I think it's five times a minute. It's quite mean. And so you would only be able to call for a share price five times. But it turns out that it also only counts it as one call if you make a call and ask for all the share prices in the market, a snapshot of every share price. So I've done something. If you ask for a share price like Apple and you're on the free plan, it will call for all share prices if we use that one call. And then it will cache that in memory. And the subsequent times you call this, it will just return from the cache.

I've made a python module `market.py` that uses this API to look up share prices.

But the free API is quite heavily rate limited - so I've been a bit sneaky; when you ask for a share price, this function retrieves the entire end-of-day equity market, and caches it in our database.

`market.py`:

Let me show you that for a second. If we look in market.py, 
* you'll see here that there's this get share price `def get_share_price(symbol) -> float:`. I check we've got a Polygon key `def get_share_price_polygon(symbol) -> float:`. 
* And then I call this get share price Polygon. And I see if you're on the paid plan or not. 
* If you're on the free plan, I call this function `def get_share_price_polygon_min(symbol) -> float:`, if you're following me, and this function is basically going to end up getting the full market for the prior business day  `@lru_cache(maxsize=2)
def get_market_for_prior_date(today):`, and it will cache that. So if this has been called before with the same date, it will just return the same full market. 

So it's just a nice way of making sure that we can always get that share price and you won't hit the rate limits.


---

Let me show you that. If we get the share price for Apple, it's $195. And now I can put that in a tight loop and get it 1,000 times, and we simply get $195. And I assure you that if this weren't being plucked from the cache, then we would have problems with that. We would hit the rate limit immediately. So that's just something to understand, just something that I've put in there.


In [13]:
from market import get_share_price
get_share_price("AAPL")

232.78

In [14]:
# no rate limiting concerns!

for i in range(1000):
    get_share_price("AAPL")
get_share_price("AAPL")

232.78

### And I've made this into an MCP Server

Just as we did with accounts.py; see `market_server.py`

In [15]:
# Ver el contenido del archivo
with open("market_server.py", "r") as f:
    content = f.read()
    print("Content of market_server.py:")
    print(content)

Content of market_server.py:
# from mcp.server.fastmcp import FastMCP
# from market import get_share_price

# mcp = FastMCP("market_server")

# @mcp.tool()
# async def lookup_share_price(symbol: str) -> float:
#     """This tool provides the current price of the given stock symbol.

#     Args:
#         symbol: the symbol of the stock
#     """
#     return get_share_price(symbol)

# if __name__ == "__main__":
#     mcp.run(transport='stdio')


#!/usr/bin/env python3
import json
import sys
from market import get_share_price

def handle_request(request):
    """Handle MCP requests"""
    method = request.get("method")
    
    if method == "initialize":
        # Handle initialization request
        return {
            "jsonrpc": "2.0",
            "id": request.get("id"),
            "result": {
                "protocolVersion": "2024-11-05",
                "capabilities": {
                    "tools": {}
                },
                "serverInfo": {
                    "nam

In [16]:
params = {"command": "python", "args": ["market_server.py"]}
async with MCPServerStdio(params=params) as server:
    mcp_tools = await server.list_tools()
mcp_tools

[Tool(name='lookup_share_price', description='Get the current price of a stock symbol', inputSchema={'type': 'object', 'properties': {'symbol': {'type': 'string', 'description': 'Stock symbol (e.g., AAPL)'}}, 'required': ['symbol']}, annotations=None)]

### Let's try it out!

Hopefully gpt-4o-mini is smart enough to know that the symbol for Apple is AAPL

In [17]:
instructions = "You answer questions about the stock market."
request = "What's the share price of Apple?"
model = "gpt-4.1-mini"

async with MCPServerStdio(params=params) as mcp_server:
    agent = Agent(name="agent", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("conversation"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

The current share price of Apple (AAPL) is $232.78.

## Polygon.io Part 2: Paid Plan - Totally Optional!

If you are interested, you can subscribe to the monthly plan to get more up to date market data, and unlimited API calls.

If you do wish to do this, then it also makes sense to use the full MCP server that Polygon.io has released, to take advantage of all their functionality.

---


**market\_server.py – Local Stock Price Server**
Purpose: Provides stock price lookup tool via MCP protocol
What it does:

* Uses FastMCP to create a local MCP server
* Calls functions from market.py
  Tool: lookup\_share\_price(symbol) – retrieves current stock price for any symbol
  Status: WORKING – Fast, reliable, no network issues

**market.py – Stock Data Handler**
Purpose: Core module that fetches stock prices from Polygon.io API
What it does:

* Free plan: Retrieves end-of-day prices (cached to avoid rate limits)
* Paid plan: Retrieves 15-minute delayed prices
* Fallback: Generates random prices if API fails
  Status: WORKING – Successfully fetches AAPL price (\$232.78)

**test\_market\_server.py – Local Server Tester**
Purpose: Tests whether the local market server works correctly
What it does:

* Connects to the local MCP server
* Lists available tools
  Status: WORKING – Confirmed server has lookup\_share\_price tool

**test\_polygon\_server.py – Remote Server Tester**
Purpose: Tests connection to Polygon.io's official MCP server
What it does:

* Connects to remote server with a 60-second timeout
* Lists all 29 available tools
  Status: WORKING – Needs longer timeout (60s instead of 5s)

**Key Takeaway**
Use longer timeouts for remote MCP servers

```py
# ❌ Too short - will timeout
async with MCPServerStdio(params=params) as server:

# ✅ Better - gives server time to initialize  
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
```

Or use the local server for reliability:

```py
params = {"command": "python", "args": ["market_server.py"]}
async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as server:
```

--- 

But that's not the interesting part of using Polygon. The interesting part is if you have a paid plan and you want their MCP server with all of the capabilities. And so I'm going to show this to you just so you get a sense for it and totally optional, but if you wish you could sign up even just for the first month to experience this for yourself. So here are the parameters that we're providing for our MCP tool. It is UVX, so it's a Python-based MCP server. And you can see, interestingly, that the way it works is that you can provide a link to the GitHub repo which contains this MCP server. And you can use that. So we're calling MCP Polygon from this repo. So it doesn't have to be something that's like pip installable. It can be something that is just straight from a repo. And obviously if you do something like this, then you need to do your research. You need to go and check that this genuinely is the official GitHub repo for Polygon, the company behind Polygon.io, and make sure you're comfortable with the community traction, the number of stars, the kind of support it has, and so on. Just as you would do if you were going to clone somebody else's repo. The same kind of due diligence exactly. And so I have done that, of course, and I'm satisfied with this myself, but you should too. And then I'm passing in my Polygon key. And then I'm going to ask what tools do you provide me, this MCP server.


So we'll run this. And wowza, there's a lot of them. So there's a bunch of different tools. There's a lot that the model is going to be allowed to do. It includes doing things like getting the last trade. It has crypto data in here as well as FX data, getting market status, tickers, dividends, conditions, financials. There's a lot to get here. And so this is something which equips our agent with lots of capabilities to analyze financial markets directly. Now, you can give all of these tools to your agent, even if you're on the free plan. But if you do that, most of these will respond and say this tool isn't available for people on the free plan. So it's a bit of a bore. What you have to do is either just only provide the one tool, as I did for the free plan example, or sign up for the pay plan.

In [20]:

params = {"command": "uvx",
          "args": ["--from", "git+https://github.com/polygon-io/mcp_polygon@v0.1.0", "mcp_polygon"],
          "env": {"POLYGON_API_KEY": polygon_api_key}
          }
async with MCPServerStdio(params=params) as server:
    mcp_tools = await server.list_tools()
mcp_tools


[Tool(name='get_aggs', description='\n    List aggregate bars for a ticker over a given date range in custom time window sizes.\n    ', inputSchema={'properties': {'ticker': {'title': 'Ticker', 'type': 'string'}, 'multiplier': {'title': 'Multiplier', 'type': 'integer'}, 'timespan': {'title': 'Timespan', 'type': 'string'}, 'from_': {'anyOf': [{'type': 'string'}, {'type': 'integer'}, {'format': 'date-time', 'type': 'string'}, {'format': 'date', 'type': 'string'}], 'title': 'From'}, 'to': {'anyOf': [{'type': 'string'}, {'type': 'integer'}, {'format': 'date-time', 'type': 'string'}, {'format': 'date', 'type': 'string'}], 'title': 'To'}, 'adjusted': {'anyOf': [{'type': 'boolean'}, {'type': 'null'}], 'default': None, 'title': 'Adjusted'}, 'sort': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'default': None, 'title': 'Sort'}, 'limit': {'anyOf': [{'type': 'integer'}, {'type': 'null'}], 'default': None, 'title': 'Limit'}, 'params': {'anyOf': [{'additionalProperties': True, 'type': 'object'

### Wow that's a lot of tools!

Let's try them out - hopefully the sheer number of tools doesn't overwhelm gpt-4o-mini!

With the $29 monthly plan, we don't have access to some of the APIs, so I've needed to specify which APIs can be called.

If you've splashed out on a bigger plan, feel free to remove my extra constraint..


Okay. So let's try these out. So as I said, because if you're on the free plan, we have to be specific that it should only use the get snapshot ticker tool. And so I will run this and have it hopefully use the right tool. So we're now equipping the agent with all of those tools that we just saw. And oh, and it hasn't used the right tool. That's super interesting. Actually, let me also, I'm going to upgrade the model while I'm doing it. Even though I said get snapshot ticker, let's try it a second time. It's always good to see agents being unpredictable. Let's see whether it gets it right the second time and uses the right ticker. It does. So I attempted to go and re-record this, but I won't because I think it's important to see that with agentic AI, you do get this sometimes ill behavior when it doesn't correctly follow the prompts. And you need to be able to account for that in what you do. And also, I don't know if this is what fixed it, but upgrading to a more recent model did appear in this case to make it perform better. But now it did use the right tool and it got the latest share price at $195. 

In [21]:
instructions = "You answer questions about the stock market."
request = "What's the share price of Apple? Use your get_snapshot_ticker tool to get the latest price."
model = "gpt-4.1-mini"

async with MCPServerStdio(params=params) as mcp_server:
    agent = Agent(name="agent", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("conversation"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

I am currently unable to access the latest share price of Apple due to authorization restrictions. If you have another request or if there's anything else you'd like to know, please let me know!

## Setting up your .env file

If you do decide to have a paid plan, please add this to your .env file to indicate:

`POLYGON_PLAN=paid`

And if you decide to go all the way for the realtime API, then please do:

`POLYGON_PLAN=realtime`

Okay. So if you do have a paid plan, then you can now add this to your M file. You just have to, in your .env file, say polygon plan equals paid that makes sure that I switch it to use the right tools. And if you do decide to go all the way and have the premium, a real-time plan, you can set this to real-time and I'll be sure to use all of the real-time APIs. But I have got just this paid plan. So I'm going to go and update my M file right now. Okay. So now if I now run this cell, it should confirm that I'm on the right plan. Let's see. Yep. You've chosen to subscribe to the paid polygon plan. And so we'll be looking at prices on a 15-minute delay. And that, for the time being, is the foray into MCP servers. But there are so many more and that's where I will get to with the exercises.

In [22]:
load_dotenv(override=True)

polygon_plan = os.getenv("POLYGON_PLAN")
is_paid_polygon = polygon_plan == "paid"
is_realtime_polygon = polygon_plan == "realtime"

if is_paid_polygon:
    print("You've chosen to subscribe to the paid Polygon plan, so the code will look at prices on a 15 min delay")
elif is_realtime_polygon:
    print("Wowzer - you've chosen to subscribe to the realtime Polygon plan, so the code will look at realtime prices")
else:
    print("According to your .env file, you've chosen to subscribe to the free Polygon plan, so the code will look at EOD prices")

According to your .env file, you've chosen to subscribe to the free Polygon plan, so the code will look at EOD prices


## And that's it for today!

I've removed the part of this lab that uses the "Financial Datasets" mcp server, because it's inferior - more expensive with fewer APIs.

And this way we get to use the same provider for Free and Paid APIs.

But if you want to see the code, just look in the git history for a prior version.


So I would suggest that you go onto the marketplaces and look for tools which interest you and experiment with them right here. Try and use, I say here, all three approaches. You'll probably find, like me, that there actually aren't easily available approaches with remote MCP servers. So you might not want to go there unless you are already paying user of something like Asana. But assuming that you're not, then go with those first two approaches. Find examples of MCP servers that will run locally and only stay locally, do things on your local computer. Find examples that call the web. And find examples of MCP servers that are Python-based and those that are JavaScript-based. And try them all out. Experiment with them and have fun giving capabilities to your agents and seeing them taking advantage of them. Well, hopefully you took me up on that. You did some exercises and you've added more MCP servers. And that's a wrap on day three of week six. And that means that we're about to head into the capstone project of building a trading floor. And I can't wait to show you this. It's such a cool project. I'll see you next time.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td>
            <h2 style="color:#ff7800;">Exercises</h2>
            <span style="color:#ff7800;">Explore MCP server marketplaces and integrate your own, using all 3 approaches.
            </span>
        </td>
    </tr>
</table>